In [ ]:
import optuna
from datetime import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed
from modules.StrategySearcher import StrategySearcher
import warnings
import os
warnings.filterwarnings("ignore")

configs = [
    dict(
        symbol='XAUUSD', timeframe='H1', direction='both', pruner_type='successive',
        train_start=datetime(2020,2,1), train_end=datetime(2025,2,1),
        test_start=datetime(2022,2,1),  test_end=datetime(2023,2,1),
        search_type='clusters', search_subtype='lgmm', search_filter='',
        label_method='trend', n_models=1, debug=True,
    ),
]

# Crear tag para cada configuración
for cfg in configs:
    # Construir el tag asegurando que no haya dobles guiones bajos por campos vacíos
    tag_parts = [
        cfg['symbol'],
        cfg['timeframe'],
        cfg['direction'],
        cfg['label_method'][:2],
        cfg['search_type'][:3],
        (cfg.get('search_subtype') or '')[:2],
        (cfg.get('search_filter') or '')[:2],
    ]
    # Filtrar partes vacías y unir con "_"
    cfg["tag"] = "_".join([part for part in tag_parts if part]).strip("_")

DB_FILE = f"optuna_dbs/{cfg['tag']}.db"
DB_PATH = f"sqlite:///{DB_FILE}"
STUDY_NAME = f"{cfg['tag']}"

study = None
if not os.path.exists(DB_FILE):
    study = None
else:
    try:
        study = optuna.load_study(study_name=STUDY_NAME, storage=DB_PATH)
    except Exception:
        study = None

n_trials = 5000
if study:
    n_trials = len(study.trials) + n_trials

for cfg in configs:
    cfg['n_trials'] = n_trials

def launch(cfg):
    s = StrategySearcher(**cfg)
    s.run_search()

with ProcessPoolExecutor(max_workers=len(configs)) as pool:
    futures = {pool.submit(launch, c): c["tag"] for c in configs}
    for f in as_completed(futures):
        tag = futures[f]
        try:
            print(f"[{tag}] terminado")
        except Exception as e:
            print(f"[{tag}] falló: {e}")

🔍 DEBUG suggest_all_params - Parámetros generados:
🔍   CatBoost params: ['cat_main_iterations', 'cat_main_depth', 'cat_main_learning_rate', 'cat_main_l2_leaf_reg', 'cat_main_early_stopping', 'cat_meta_iterations', 'cat_meta_depth', 'cat_meta_learning_rate', 'cat_meta_l2_leaf_reg', 'cat_meta_early_stopping']
🔍   Feature params: ['feature_main_periods', 'feature_main_stats', 'feature_meta_periods', 'feature_meta_stats']
🔍   Label params: ['label_rolling', 'label_polyorder', 'label_threshold', 'label_vol_window', 'label_markup', 'label_min_val', 'label_max_val', 'label_atr_period', 'label_method_random']
🔍   Algo params: ['cluster_lgmm_components', 'cluster_lgmm_covariance', 'cluster_lgmm_iter', 'oof_resid_percentile', 'meta_threshold', 'main_threshold']
🔍   Total params: 29
🔍 DEBUG search_clusters lgmm - Parámetros clusters: {'cluster_lgmm_components': 9, 'cluster_lgmm_covariance': 'full', 'cluster_lgmm_iter': 365}
🔍 DEBUG: base_df.shape = (32877, 5)
🔍 DEBUG: train_start = 2020-02-01 00: